In [22]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import numpy as np
import matplotlib.pyplot as plt

In [23]:
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)
test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

Create training and test set with 500 examples of classes 5 and 8 respectively and change labels to 0 and 1. At the same time flip 30% of the labels.

In [24]:
train_set = [ex for ex in train_dataset if ex[1]==5][:500] + [ex for ex in train_dataset if ex[1]==8][:500]
test_set = [ex for ex in test_dataset if ex[1]==5][:500] + [ex for ex in test_dataset if ex[1]==8][:500]

In [25]:
train_set_01_images = []
test_set_01_images = []
train_set_01_labels = []
test_set_01_labels = []
num_flips = 500//3 #num_flips=0 to have no flips
for i,ex_tr in enumerate(train_set): # [0,499] ->5, [500,999] ->8
    ex_tr = list(ex_tr)
    if ex_tr[1]==5:
        if i<num_flips: # 0-30% of 5s(0-499) are flipped to 8s, 
            ex_tr[1]=1
            # 1 is the label for 8s
            # 0 is the label for 5s
        else: 
            ex_tr[1]=0
    else: 
        if i<num_flips+500: # 0-30% of 8s(500-999) are flipped to 5s
            ex_tr[1]=0
        else: 
            ex_tr[1]=1
    train_set_01_images.append(ex_tr[0])
    train_set_01_labels.append(ex_tr[1])
    
for ex_te in test_set:
    ex_te = list(ex_te)
    if ex_te[1]==5:
        ex_te[1]=0
    else: 
        ex_te[1]=1
    test_set_01_images.append(ex_te[0])
    test_set_01_labels.append(ex_te[1])
train_set_01_images = torch.stack(train_set_01_images)  # why stack? anwser: to make it a tensor, and the shape is (1000, 1, 28, 28)
train_set_01_images = train_set_01_images.view(-1, 28*28) # 1000, 784

test_set_01_images = torch.stack(test_set_01_images)
test_set_01_images = test_set_01_images.view(-1, 28*28)

train_set_01_labels = torch.tensor(train_set_01_labels)
test_set_01_labels = torch.tensor(test_set_01_labels) 

# shuffle training set
n_sample = len(train_set_01_images)
np.random.seed(0)
order = np.random.permutation(n_sample)
train_set_01_images = train_set_01_images[order]
train_set_01_labels = train_set_01_labels[order]

In [26]:
print("size train set:", train_set_01_images.shape)
print("size test set:", test_set_01_images.shape)

size train set: torch.Size([1000, 784])
size test set: torch.Size([1000, 784])


## logistic regression

In [27]:
# def sigmoid(X):
#     return 1/(1+torch.exp(-X))

# def f(X,theta): # x 1*784, theta 784*1
#     return sigmoid(torch.matmul(X,theta))

# def loss(X,y,theta):
#     epsilon = 1e-8 # to avoid nan
#     X = X.type(torch.float)
#     y = y.type(torch.float)
#     loss1 = -torch.matmul(y,torch.log(f(X,theta)+epsilon)) - torch.matmul((1-y),torch.log(1-f(X,theta)+epsilon))
#     return loss1

# def loss_grad(X,y,theta):
#     X = X.type(torch.float)
#     y = y.type(torch.float)
#     return torch.matmul(X.T, f(X,theta)-y).sum(1).view(-1,1)


In [28]:
# max_epochs = 10000
# step_size = 0.0000001
# theta1 = torch.ones(28*28,1)

# for i in range(max_epochs): 
#     grad = loss_grad(train_set_01_images, train_set_01_labels, theta1)
#     tmp = theta1
#     theta1 = theta1 - step_size * grad
#     print(loss(train_set_01_images, train_set_01_labels, theta1))


In [35]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.sigmoid(x)
        return x
    
net = Net()

import torch.optim as optim
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

max_epochs = 1000
X = train_set_01_images.type(torch.float)
y = train_set_01_labels.type(torch.float).view(-1,1)
for epochs in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1,1), y)
    loss.backward()
    optimizer.step()
    print(loss.item())


0.690821647644043
0.6900579929351807
0.6886357665061951
0.6866686940193176
0.6842674612998962
0.6815328598022461
0.6785529851913452
0.6754049062728882
0.6721571683883667
0.6688730716705322
0.665610134601593
0.6624193787574768
0.6593431234359741
0.656412661075592
0.653647780418396
0.6510586738586426
0.6486475467681885
0.6464121341705322
0.6443473100662231
0.6424461603164673
0.640701413154602
0.6391039490699768
0.6376432776451111
0.6363075375556946
0.6350839138031006
0.6339593529701233
0.6329211592674255
0.6319580674171448
0.6310603022575378
0.6302192211151123
0.6294276118278503
0.6286789774894714
0.6279674172401428
0.6272876262664795
0.6266348361968994
0.626004695892334
0.6253939867019653
0.6248000264167786
0.6242208480834961
0.6236551403999329
0.6231017112731934
0.6225597858428955
0.6220288872718811
0.6215083599090576
0.6209975481033325
0.6204963326454163
0.620004415512085
0.6195217967033386
0.6190482378005981
0.6185840368270874
0.6181291937828064
0.6176837086677551
0.617247462272644
0